In [11]:
%%time
import numpy as np
from numpy import exp

def mult(a, b):
    res = np.matrix(np.zeros(a.shape), dtype = np.float64)
    for i in range(a.shape[0]):
        res[i] = a[i]*b[i]
    return res

def NewtonSolver(f, u0, eps=10**-12, h = 10**-5):
    global N
    u = u0
    dx = np.matrix(np.ones(u0.shape), dtype = np.float64)
    i = 0
    fu = np.zeros( (u0.shape[0], u0.shape[0]), dtype = np.float64)
    while (sum(mult(dx,dx)).item(0))**1/2 > eps :
        print("Iteration:", i)
        i+=1
        du = np.matrix(np.zeros(u0.shape), dtype = np.float64)
        for j in range( u0.shape[0] ):
            du[j-1] = 0.0
            du[j] = h
            val = ( f(u+du) - f(u-du) )/(2.0*h)
            for k in range( u0.shape[0] ):
                fu[j][k]= val[k]
        fu = fu.T
        print(fu)
        dx = -1 * np.linalg.inv(fu) * f(u)
        u = u + dx
    return u

global N, k0, k1, u0v, uNv, x
N = 8
k0 = 1.0
k1 = 0.5
u0v = 0.0
uNv = 0.0
lambda0 = 40.0

def F(u):
    global N, k0, k1, u0v, uNv, x
    res = np.matrix( np.zeros((N-1, 1)), dtype = np.float64)
    i = 0
    res[i] = (u[i+1] - u[i]) * (k0 + k1 * (u[i]**2 + u[i+1]**2)/2.0) - u[i]*(k0 + k1 * u[i]**2/2.0) + u[-1]*u[i]*(x[i]-x[i+1])**2
    
    for i in range(1, (N+2)//2-2):
        res[i] = (u[i+1] - u[i]) * (k0 + k1 * (u[i]**2 + u[i+1]**2)/2) - (u[i] - u[i-1])*(k0 + k1 * (u[i]**2 + u[i-1]**2)/2) + u[-1]*u[i]*(x[i]-x[i+1])**2
    i = (N+2)//2 - 2
    h = (x[i+1] - x[i])
    res[i] = (2*h+u[i-1] - u[i]) * (k0 + k1 * (u[i]**2 + (2*h+u[i-1])**2)/2) - (u[i] - u[i-1])*(k0 + k1 * (u[i]**2 + u[i-1]**2)/2) + u[-1]*u[i]*h**2
        
    i = (N+2)//2 - 1
    h = (x[i+1] - x[i])
    res[i] = (u[i+1] - u[i]) * (k0 + k1 * (u[i]**2 + u[i+1]**2)/2) - (u[i] - (2*h+u[i-2]))*(k0 + k1 * (u[i]**2 + (2*h+u[i-2])**2)/2) + u[-1]*u[i]*h**2
    for i in range((N+2)//2, N-3):
        res[i] = (u[i+1] - u[i]) * (k0 + k1 * (u[i]**2 + u[i+1]**2)/2) - (u[i] - u[i-1])*(k0 + k1 * (u[i]**2 + u[i-1]**2)/2) + u[-1]*u[i]*(x[i]-x[i+1])**2
        
    i = N-3
    res[i] = (uNv - u[i]) * (k0 + k1 * u[i]**2) - (u[i] - u[i-1])*(k0 + k1 * (u[i]**2 + u[i-1]**2)/2) + u[-1]*u[i]*(x[i]-x[i+1])**2
    res[N-2] = u[N-2]
    #print('res3',res)
    return res

prev_answer = np.matrix(np.zeros((N//2, 1)))
for j in range(1,3):
    print('N:', N)
    u0 = np.matrix(np.zeros((N-1, 1)), dtype = np.float64)
    x = np.linspace(0, 1, N+1)
    for i in range( (N//2+2)//2 - 2):
        u0[2*i+1] = prev_answer[i]
    if not j==1:
        i = (N//2+2)//2 - 3
        u0[2*i+1] = u0[2*i+1-2] + 4*(x[2*i+1+1]-x[2*i+1])
    for i in range((N//2+2)//2 - 2, N//2-2):
         u0[2*i+1] = prev_answer[i]
    for i in range(2,N-3,2):
        u0[i] = (u0[i-1] + u0[i+1])/2
    u0[0] = (u0v + u0[1])/2
    u0[-2] = (uNv + u0[-3])/2
    if j==1:
        u0[-1] = lambda0
    #print(u0)
    prev_answer = NewtonSolver(F, u0)
    print('lambda:', prev_answer[-1])
    N*=2

N: 8
Iteration: 0
[[-1.375     1.        0.        0.        0.        0.        0.      ]
 [ 1.       -1.375     1.        0.        0.        0.        0.      ]
 [ 0.        1.       -1.375     1.        0.        0.        0.      ]
 [ 0.        0.        2.046875 -1.390625  0.        0.        0.      ]
 [ 0.        0.        1.046875  0.       -1.390625  1.        0.      ]
 [ 0.        0.        0.        0.        1.       -1.375     0.      ]
 [ 0.        0.        0.        0.        0.        0.        1.      ]]
Iteration: 1
[[-2.02006308e+00  1.01530648e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00 -1.95227440e-03]
 [ 1.00835453e+00 -2.03097981e+00  1.00710726e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00 -2.68437730e-03]
 [ 0.00000000e+00  1.01567333e+00 -2.01752841e+00  1.00440611e+00
   0.00000000e+00  0.00000000e+00 -1.73874439e-03]
 [ 0.00000000e+00  0.00000000e+00  2.02363860e+00 -2.00817844e+00
   0.00000000e+00  0.00000000e+00  2.936

In [ ]:
sum(mult(u0,u0)).item(0)